Приложение к итоговой работе "Метрики, гипотезы, точки роста" 

In [200]:
import pandas as pd
import numpy as np
import re

In [154]:
data = pd.read_csv('Data_retail.csv')

In [165]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850.0,United Kingdom


In [160]:
#Столбец UnitPrice приводим к десятичной дроби с "."

data['UnitPrice'] = [x.replace(',','.') for x in data['UnitPrice']]

In [166]:
#Меняем тип столбца на float
data['UnitPrice'] = data['UnitPrice'].astype(float)

In [167]:
 data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    54214 non-null  object 
 1   StockCode    54214 non-null  object 
 2   Description  54080 non-null  object 
 3   Quantity     54214 non-null  int64  
 4   InvoiceDate  54214 non-null  object 
 5   UnitPrice    54214 non-null  float64
 6   CustomerID   40643 non-null  float64
 7   Country      54214 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 3.3+ MB


In [168]:
#Проверяем датафрейм на дубли. Получаем 50 дублей
data.duplicated().value_counts()

False    54164
True        50
dtype: int64

In [169]:
#Удаляем дубли
df = data.drop_duplicates()

In [170]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/11 12:31,0.85,15804.0,United Kingdom
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/11 12:31,2.08,15804.0,United Kingdom
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/11 12:31,0.42,15804.0,United Kingdom
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/11 12:50,1.95,12680.0,France


In [192]:
#Базовые статистики
df.describe()

,Quantity,UnitPrice,CustomerID,cancellation
count,54164.000000,54164.000000,54164.000000,54164.000000
mean,9.617126,4.308856,11453.523613,0.017705
std,57.384404,47.856474,6785.572252,0.131880
min,-9600.000000,0.000000,0.000000,0.000000
25%,1.000000,1.250000,0.000000,0.000000
50%,3.000000,2.080000,14360.000000,0.000000
75%,10.000000,4.130000,16241.000000,0.000000
max,2400.000000,6930.000000,18287.000000,1.000000


In [172]:
#Вывожу строки, где вместо CustomerID стоит NaN
nan_rows = df[df.isnull().T.any().T]
nan_rows

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
158,536544,21809,CHRISTMAS HANGING TREE WITH BELL,1,12/1/10 14:32,2.51,NaN,United Kingdom
159,536544,21870,I CAN ONLY PLEASE ONE PERSON MUG,1,12/1/10 14:32,3.36,NaN,United Kingdom
160,536544,21879,HEARTS GIFT TAPE,1,12/1/10 14:32,1.66,NaN,United Kingdom
161,536544,21928,JUMBO BAG SCANDINAVIAN PAISLEY,1,12/1/10 14:32,4.21,NaN,United Kingdom
162,536544,22069,BROWN PIRATE TREASURE CHEST,1,12/1/10 14:32,3.36,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
54161,581498,48138,DOORMAT UNION FLAG,1,12/9/11 10:26,14.13,NaN,United Kingdom
54162,581498,82482,WOODEN PICTURE FRAME WHITE FINISH,4,12/9/11 10:26,4.96,NaN,United Kingdom
54163,581498,84032A,CHARLIE+LOLA PINK HOT WATER BOTTLE,4,12/9/11 10:26,5.79,NaN,United Kingdom
54164,581498,84558a,3D DOG PICTURE PLAYING CARDS,1,12/9/11 10:26,5.79,NaN,United Kingdom


In [173]:
#Процент «NaN» в каждом столбце
df.isnull().sum()/len(df)*100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.247397
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     25.055387
Country         0.000000
dtype: float64

In [174]:
#Заменяю NaN на 0 в столбце CustomerID
df['CustomerID'] = df['CustomerID'].fillna(0)

<ipython-input-174-69f9ec357452>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustomerID'] = df['CustomerID'].fillna(0)


In [175]:
#Преобразовываю значения в формат int
df['CustomerID'] = df['CustomerID'].astype(np.int64)

<ipython-input-175-14ebab5930f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustomerID'] = df['CustomerID'].astype(np.int64)


In [185]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancellation
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,0
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047,United Kingdom,0
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047,United Kingdom,0
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047,United Kingdom,0
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850,United Kingdom,0
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/11 12:31,0.85,15804,United Kingdom,0
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/11 12:31,2.08,15804,United Kingdom,0
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/11 12:31,0.42,15804,United Kingdom,0
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/11 12:50,1.95,12680,France,0


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54164 entries, 0 to 54213
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    54164 non-null  object 
 1   StockCode    54164 non-null  object 
 2   Description  54030 non-null  object 
 3   Quantity     54164 non-null  int64  
 4   InvoiceDate  54164 non-null  object 
 5   UnitPrice    54164 non-null  float64
 6   CustomerID   54164 non-null  int64  
 7   Country      54164 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.7+ MB


In [ ]:
#Количество отрицательных значений в столбце Quantity 

In [178]:
df[df['Quantity'] <0].shape[0]

1082

In [184]:
# Количество заказов с ценой товара =0
df[df['UnitPrice'] == 0].shape[0]

225

In [193]:
# Удаляем заказы с ценой 0
df_del = df[df['UnitPrice'] != 0]
df_del

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancellation
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,0
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047,United Kingdom,0
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047,United Kingdom,0
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047,United Kingdom,0
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850,United Kingdom,0
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,12/9/11 12:31,0.85,15804,United Kingdom,0
54210,581585,23084,RABBIT NIGHT LIGHT,12,12/9/11 12:31,2.08,15804,United Kingdom,0
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,12/9/11 12:31,0.42,15804,United Kingdom,0
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/11 12:50,1.95,12680,France,0


In [197]:
#Считаем количество отмененных заказов
df_del['Cancellation']=df_del.InvoiceNo.str.extract('([C])').fillna(0).replace({'C':1})
df_del['Cancellation'].value_counts()

<ipython-input-197-2579738c4a09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_del['Cancellation']=df_del.InvoiceNo.str.extract('([C])').fillna(0).replace({'C':1})


0    52980
1      959
Name: Cancellation, dtype: int64

In [198]:
df_del.head(50)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancellation,Cancellation
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850,United Kingdom,0,0
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047,United Kingdom,0,0
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047,United Kingdom,0,0
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047,United Kingdom,0,0
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850,United Kingdom,0,0
5,536375,82494L,WOODEN FRAME ANTIQUE WHITE,6,12/1/10 9:32,2.55,17850,United Kingdom,0,0
6,536378,21033,JUMBO BAG CHARLIE AND LOLA TOYS,10,12/1/10 9:37,2.95,14688,United Kingdom,0,0
7,536381,82567,"AIRLINE LOUNGE,METAL SIGN",2,12/1/10 9:41,2.10,15311,United Kingdom,0,0
8,536381,21169,YOU'RE CONFUSING ME METAL SIGN,3,12/1/10 9:41,1.69,15311,United Kingdom,0,0
9,536381,21175,GIN + TONIC DIET METAL SIGN,2,12/1/10 9:41,2.10,15311,United Kingdom,0,0


In [199]:
df_del.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53939 entries, 0 to 54213
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   InvoiceNo     53939 non-null  object 
 1   StockCode     53939 non-null  object 
 2   Description   53939 non-null  object 
 3   Quantity      53939 non-null  int64  
 4   InvoiceDate   53939 non-null  object 
 5   UnitPrice     53939 non-null  float64
 6   CustomerID    53939 non-null  int64  
 7   Country       53939 non-null  object 
 8   cancellation  53939 non-null  int64  
 9   Cancellation  53939 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 4.5+ MB
